## Demonstration of Model Selection in Multiclass Classification

This jupyter notebook is part of a [collection of notebooks](../index.ipynb) on various topics of Data-Driven Signal Processing. Please direct questions and suggestions to [Sascha.Spors@uni-rostock.de](mailto:Sascha.Spors@uni-rostock.de).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from keras_tuner import HyperParameters, RandomSearch

In [2]:
N = 2000  # total number of examples
C = 3  # number of classes

X, Y = make_classification(n_samples=N,
                           n_classes=C,
                           n_clusters_per_class=1,
                           n_features=2,
                           n_redundant=0,
                           n_informative=2,
                           random_state=42)

Y1h = to_categorical(Y, C)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y1h, test_size=0.2)

In [3]:
def build_model(hp):
    
    model = Sequential()
    model.add(Input(shape=(2,)))
    
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(Dense(units=hp.Int(f"units_l{i}", min_value=16, max_value=128, step=16),
                        activation=hp.Choice("activation", ["relu", "tanh"])))
        
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.25))
    
    model.add(Dense(C, activation="softmax"))
    
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [4]:
build_model(HyperParameters())

tuner = RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=20,
    executions_per_trial=2,
    overwrite=True,
    directory="tuner_results",
    project_name="multiclass",
)

2022-01-27 17:59:10.778137: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_l0 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [6]:
stop_early = EarlyStopping(monitor='val_accuracy', patience=5)

tuner.search(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), callbacks=[stop_early])

Trial 20 Complete [00h 00m 02s]
val_accuracy: 0.9399999976158142

Best val_accuracy So Far: 0.9399999976158142
Total elapsed time: 00h 01m 02s
INFO:tensorflow:Oracle triggered exit


In [8]:
tuner.results_summary()

Results summary
Results in tuner_results/multiclass
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 3
units_l0: 32
activation: relu
dropout: False
lr: 0.008690166278024544
units_l1: 96
units_l2: 80
Score: 0.9399999976158142
Trial summary
Hyperparameters:
num_layers: 2
units_l0: 128
activation: relu
dropout: False
lr: 0.0042198127756051884
units_l1: 112
units_l2: 80
Score: 0.9387499988079071
Trial summary
Hyperparameters:
num_layers: 1
units_l0: 128
activation: relu
dropout: True
lr: 0.005553415568099142
units_l1: 80
units_l2: 64
Score: 0.9387499988079071
Trial summary
Hyperparameters:
num_layers: 3
units_l0: 96
activation: relu
dropout: False
lr: 0.0009031111745904648
units_l1: 80
units_l2: 112
Score: 0.9387499988079071
Trial summary
Hyperparameters:
num_layers: 3
units_l0: 128
activation: tanh
dropout: False
lr: 0.0026490704262329893
units_l1: 32
units_l2: 32
Score: 0.9350000023841858
Trial summary
Hyperparameters:
num_

In [9]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]

best_model.build()
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                96        
                                                                 
 dense_1 (Dense)             (None, 96)                3168      
                                                                 
 dense_2 (Dense)             (None, 80)                7760      
                                                                 
 dense_3 (Dense)             (None, 3)                 243       
                                                                 
Total params: 11,267
Trainable params: 11,267
Non-trainable params: 0
_________________________________________________________________


**Copyright**

This notebook is provided as [Open Educational Resource](https://de.wikipedia.org/wiki/Open_Educational_Resources).
The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/)
, the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: Sascha Spors, Data driven audio signal processing - Lecture supplementals.